# Analysis

This notebook provides several analyses to gain deeper intuition into system performance.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import pandas as pd
import pickle
import soundfile as sf
import IPython.display as ipd
import os.path
import import_ipynb
import system_utils

## Visualizing errors over time

The first analysis is to simply visualize the magnitude of alignment errors over time for a specific concerto movement.

In [ ]:
def visualize_errors_over_time(eval_dir, scenarios_summary, fullmix_id):
    '''
    Visualize the magnitude of alignment error over time for a given full mix recording.  
    Measures that are not evaluated will be displayed as having zero error.
    
    Inputs
    eval_dir: the eval directory containing alignment error information
    scenarios_summary: filepath specifying the scenarios.summary file
    fullmix_id: id specifying the full mix recording of interest, e.g. 'rach2_mov1_PO2'
    '''

    # parse relevant files
    with open(f'{eval_dir}/errs.pkl', 'rb') as f:
        errors_info = pickle.load(f) # key: scenario_id, value: (errs, measNums)
    scenarios_info = system_utils.get_scenario_info(scenarios_summary) # key: scenario_id, value: dict with scenario info
    
    # aggregate data to display
    data = {} # key: tsm_factor, value: dict whose key is the measure number and whose value is the align error
    for scenario_id in scenarios_info:
        if fullmix_id in scenarios_info[scenario_id]['po']:
            tsm_factor = scenarios_info[scenario_id]['p'].split('/')[-2] # e.g. 'tsm0.80'
            errs, measNums = errors_info[scenario_id] # only contains evaluation measures
            if tsm_factor not in data:
                data[tsm_factor] = {}
            for err, measNum in zip(errs, measNums):
                data[tsm_factor][measNum] = err
    
    # display non-evaluated measures as having 0 error
    for tsm_factor in data:
        maxMeasNum = np.max(list(data[tsm_factor].keys()))
        for i in range(maxMeasNum):
            if i not in data[tsm_factor]:
                data[tsm_factor][i] = 0
   
    # show alignment error vs measure number
    fig, axs = plt.subplots(len(data.keys()), sharex=True, sharey=True)
    for i, tsm_factor in enumerate(data):
        measures = sorted(data[tsm_factor].keys())
        errors = [data[tsm_factor][m] for m in measures]
        axs[i].plot(measures, errors)
        axs[i].set_title(f'{fullmix_id} - {tsm_factor}')
        axs[i].grid(linestyle='--')
    plt.xlabel('Measure Number')
    plt.ylabel('Alignment Error (sec)')
    plt.ylim([-2.5, 2.5])
    
    return data

In [ ]:
EVAL_DIR = 'eval/offlineDTW' # eval directory to visualize
SCENARIOS_SUMMARY = 'scenarios/scenarios.summary'

In [ ]:
d = visualize_errors_over_time(EVAL_DIR, SCENARIOS_SUMMARY, fullmix_id = 'rach2_mov1_PO1')

## Generating sonifications

The second analysis is to use the estimated alignment to time-scaled modify the orchestral recording.  We generate an audio file that contains the original piano recording on the left channel and the time-scale modified accompaniment on the right channel.

In [ ]:
import import_ipynb
import tsm_tools

In [ ]:
def get_preprocessed_wp(align_file, hop_len, downsample, dur1, dur2):
    '''
    Performs preprocessing on a given warping path to ensure that the time-scale modification
    can be carried out.
    
    Inputs
    align_file: filepath to the .npy file specifying the warping path (in frames) between the two audio files
    hop_len: specifies the hop length in seconds between frames, needed to convert warping path to timestamps
    downsample: downsample the warping path by this factor to smooth out the TSM
    dur1: the duration of file1 (seconds)
    dur2: the duration of file2 (seconds)
    
    Returns the modified warping path.
    '''
    wp = np.load(align_file) # 2xN array specifying file1-file2 alignment (in frames)
    wp = wp[:,0::downsample] * hop_len # downsample for smoothing, convert to sec    
    if not np.array_equal(wp[:,0], [0,0]):
        wp = np.hstack((np.array([0,0]).reshape((2,-1)), wp)) # alignment must start at (0,0)
    wp = np.hstack((wp, np.array([dur1, dur2]).reshape((2,-1)))) # alignment must extend to ends    
    return wp

In [ ]:
def mix_separate_channels(left_channel, right_channel):
    '''
    Merges two mono audio waveforms into a stereo audio waveform.  If the two waveforms differ
    in length, the longer of the two is truncated.
    
    Inputs
    left_channel: the audio waveform for the left channel
    right_channel: the audio waveform for the right channel
    '''
    N = min(len(left_channel), len(right_channel))
    mixed = np.zeros((N, 2))
    mixed[:,0] = left_channel[0:N]
    mixed[:,1] = right_channel[0:N]
    return mixed

In [ ]:
def sonifyWithAccompaniment(audiofile1, audiofile2, align_file, downsample, hop_len, outfile = None):
    '''
    Generates a stereo audio recording with one audio recording on the left channel and another audio
    recording on the other channel, where time-scale modification has been applied to the latter so
    that the two recordings are appropriately synchronized.
    
    Inputs
    audiofile1: filepath to the audio recording that will remain unmodified
    audiofile2: filepath to the audio recording that will be time-scaled modified
    align_file: filepath to the .npy file specifying the warping path (in frames) between the two audio files
    downsample: downsample the warping path by this factor to smooth out the TSM
    hop_len: specifies the hop length in seconds between frames, needed to convert warping path to timestamps
    outfile: the output audio file to generate
    '''
    y1, sr = lb.load(audiofile1)
    y2, sr = lb.load(audiofile2)
    wp = get_preprocessed_wp(align_file, hop_len, downsample, len(y1)/sr, len(y2)/sr) # file1-file2 alignment in sec
    y2_tsm = tsm_tools.tsmvar_hybrid(y2, np.flipud(wp))
    y_mixed = mix_separate_channels(y1, y2_tsm)
    if outfile:
        sf.write(outfile, y_mixed, sr, subtype='PCM_16')
    return y_mixed

Here is example usage for sonifying a single scenario:

In [ ]:
audiofile1 = f'scenarios/s1/p.wav'
audiofile2 = 'scenarios/s1/o.wav'
align_file = 'experiments/simpleOfflineDTW2/s2/hyp.npy' # p-o alignment
outfile = 'test.wav'
downsample = 20
sr = 22050
hop_len = 512./sr
y = sonifyWithAccompaniment(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

In [ ]:
# modified example
audiofile1 = f'scenarios/s1/p.wav'
audiofile2 = 'scenarios/s1/po.wav'
#align_file = 'experiments/offlineDTW/cache/rach2_mov1_O1_PO1/po_o_align.npy' # po-o alignment
align_file = 'experiments/offlineDTW/s1/p_po_align.npy' # p-po alignment
outfile = 'test.wav'
downsample = 20
sr = 22050
hop_len = 512./sr
y = sonifyWithAccompaniment(audiofile1, audiofile2, align_file, downsample, hop_len)

In [ ]:
y[:,0] = y[:,0]/3
sf.write(outfile, y, sr, subtype='PCM_16')

In [ ]:
def sonifyWithAccompaniment_batch(scenarios_dir, exp_dir, downsample, hop_len, outdir):
    '''
    Generates stereo recordings of piano (left channel) and time-scale modified orchestra recordings
    (right channel) for all scenarios.
    
    Inputs
    scenarios_dir: directory containing all scenario directories
    exp_dir: directory containing all the hypothesis alignments
    downsample: downsample the warping path by this factor to smooth out the TSM
    hop_len: specifies the hop length in seconds between frames, needed to convert warping path to timestamps
    outdir: directory to put the generated audio files
    '''
    assert not os.path.exists(outdir)
    os.makedirs(outdir)
    
    scenario_ids = list(pd.read_csv(f'{scenarios_dir}/scenarios.summary', header=None, sep=' ')[0])
    for scenario_id in scenario_ids:
        piano_file = f'{scenarios_dir}/{scenario_id}/p.wav'
        orch_file = f'{scenarios_dir}/{scenario_id}/o.mp3'
        align_file = f'{exp_dir}/{scenario_id}/hyp.npy'
        out_file = f'{outdir}/{scenario_id}.wav'
        sonifyWithAccompaniment(piano_file, orch_file, align_file, downsample, hop_len, out_file)
        
    return

The following two cells generate sonifications for all scenarios:

In [ ]:
SCENARIOS_DIR = 'scenarios'
EXP_DIR = 'experiments/simpleOfflineDTW2'
SONIFY_DIR = f'{EXP_DIR}/sonify'
downsample = 20
hop_len = 512./22050

In [ ]:
sonifyWithAccompaniment_batch(SCENARIOS_DIR, EXP_DIR, downsample, hop_len, SONIFY_DIR)

Listen to a selected example:

In [ ]:
ipd.Audio(f'{SONIFY_DIR}/s1.wav')